# Symbiote's IPC Server

One of my contributions to the Symbiote project is what we call the IPC server. Normally, symbiote system users might think that any application entering a risky “elevated” state is unsafe and concerning. The solution to this is our IPC server: a single centralized approved application that runs in a proven-to-be-safe elevated state. <br>
It has two main purposes: security and optimization.

## Security
 
This model allows a whitelisted admin-approved symbiote application running in an elevated supervisor mode transparently improve performance and efficiency of other applications. This allows user applications to not worry about crashing the entire system or messing something up as the ipc server handles incorrect input and safely performs the necessary privileged operations.<br>
Additionally, for security-intensive systems, it might be unwanted to let normal users run privileged code, so the solution to that is running a single whitelisted symbiote daemon that handles shortcutting requests accordingly. It would also allow the community to test it along with other symbiote tools and help improve its security features.<br>
The admin can also enable or disable the ipc server like a module, modify it's configured behavior, and/or change number of threads dedicated to it.


## Optimization

Whenever a user application would like to perform a shortcutted system call, our ipc interposer library transparently intercepts a normal system call and sends a request through shared memory to the ipc server. The server, in turn, performs the shortcutted version of the system call and returns the result back to the user application. This allows applications to gain essentially free speedups in performance-critical paths, since a shortcutted server might yield better performance and cacheing, and knows how to execute privileged supervisor instructions safely without crashing.

### Note
This approach is also ABI compatible, meaning no code modifications are required for original binaries of user applications!

In [ ]:
import os

'''
In here, you will learn how to use our ipc server infrastructure in your client applications.
First, you have to launch the IPC server with a certain number of threads. It will be like a running daemon on your system
and will accept requests from various client user applications, one of which you are going to build right now.
'''

# Save the full path to the directory with ipc binaries into a variable
IPCDIR = os.path.expanduser('~/') + 'Symbi-OS/Tools/bin/ipc'

# Launch the ipc server and make it a background process.
# *Note* you could also launch the ipc server using the shortcut.sh tool, but we are going to 
# run this demo with a lowered server, the functionality is equivalent and no changes have to be made.
os.system(f'{IPCDIR}/server 1 &')

In order to make use of our IPC server infrastructure, each client upon connecting to the server
will receive a "job request buffer" which it will use to make any type of request to the server.

This is how the structure looks like in C:
```
typedef struct JobRequestBuffer {
    int pid;              // Client's PID
    int cmd;              // Job command requested by the client
    int arg1;             // First integer argument
    int arg2;             // Second integer argument
    int response;         // Response from the server
    char buffer[4096];    // Command buffer
    int buffer_len;       // Commabd buffer length
    int status;  // Flag indicating which stage the job is at
    int lock;
} JobRequestBuffer_t;
```

In [ ]:
import ctypes

# We will now define the job request buffer structure in Python:
class JobRequestBuffer(ctypes.Structure):
     _fields_ = [
         ('pid', ctypes.c_int),
         ('cmd', ctypes.c_int),
         ('arg1', ctypes.c_int),
         ('arg2', ctypes.c_int),
         ('response', ctypes.c_int),
         ('buffer', ctypes.c_char * 4096),
         ('buffer_len', ctypes.c_int),
         ('status', ctypes.c_int),
         ('lock', ctypes.c_int)
    ]

All of the IPC client/server functionality is incorporated into an ```ipc.so``` shared library.

In [ ]:
# Import the ipc.so library
ipclib = ctypes.cdll.LoadLibrary(f'{IPCDIR}/ipc.so')

# For clarity, mark the return types of the functions we are going to use
ipclib.ipc_get_job_buffer.restype = ctypes.c_void_p
ipclib.submit_job_request.restype = None
ipclib.wait_for_job_completion.restype = None
ipclib.disconnect_job_buffer.restype = None

Now is one of the most important parts: connecting to the ipc server.
We can call ```ipc_get_job_buffer()``` to get a pointer to a ```JobRequestBuffer```

In [ ]:
# By requesting a job buffer, the client creates a connection with an ipc server,
# which in turn allocates a request buffer for our client in the shared memory region.
buffer_address = ipclib.ipc_get_job_buffer()

# Print the buffer address to ensure it's valid (or at least looks right)
print(f'Job buffer: {hex(buffer_address)}')

# Now that we have an address, we can construct a valid Python struct object from it
buffer = JobRequestBuffer.from_address(buffer_address)

Now, to demonstrate how the ipc server works, we will make a request for a ```write()``` system call.
The idea is essentially describing to the server what system call and its parameters you are trying to
perform and delegating the actual work to the server rather than executing the system call yourself.

In order to make a ```write()``` system call request, we need to fill the job request buffer out with the appropriate
information such as our process's pid, command we are requesting, and write call parameters: fd, buffer, len.

In [ ]:
# Define some constants for clarity
CMD_WRITE = 1
STDOUT = 1

buffer.pid = os.getpid()
buffer.cmd = CMD_WRITE
buffer.arg1 = STDOUT
buffer.buffer = b'IPC Demo Working!\n' # Text we are printing out
buffer.buffer_len = 18;

Now that we've filled out and finalized our job request buffer, we are ready
to submit the job.

The expected result is to see the message 'IPC Demo Working!' printed out to console.

In [ ]:
# Mark the job buffer as "JOB_REQUESTED" and let the ipc server pick up the job
ipclib.submit_job_request(ctypes.byref(buffer));

# Now we just sit and wait for the job to be completed and control to be returned back to us :)
ipclib.wait_for_job_completion(ctypes.byref(buffer));

One last step would be to disconnect our client from the ipc server once we are done
and no more requests will be made.

In [ ]:
# Close the ipc connection
ipclib.disconnect_job_buffer(ctypes.byref(buffer))

To demonstrate, you can also shutdown the ipc server 'remotely' through
the ```server_killer``` program. You just simply call it with the number of ipc threads to shutdown.

In [ ]:
# Shutdown the IPC server
os.system(f'{IPCDIR}/server_killer 1')